In [2]:
#!/usr/bin/env python3
### Filename: auto_mv_files.ipynb

"""
An automation script to categorize raw data captured in each experiment.

This script can be divided into four parts:
    User Settings: Modify settings to fit your experiment data.
    Stage 0:       Check files to make sure the numbers of each type of files are reasonable.
    Stage 1:       Categorize files in chronological order
    Stage 2:       Categorize files/directories with experiment names

Before you go to stage 1 & 2, 
you need to upload (manually or using py-script) all the raw data to database into specific directories, 
and remove the redundant files manually.

Author: Yuan-Jye Chen
Update: Yuan-Jye Chen 2022/10/05
"""

"""
    Future Development Plan
        (1) Do experiment with modems (at_log) and check if the script needs to be modified.
        (2) Write a script to transfer data via wi-fi automatically.
    
"""
import os
import sys
import shutil
from genericpath import exists
from pprint import pprint

# --------------------- Util Functions ---------------------
def makedir(dirpath, mode=0):  # mode=1: show message, mode=0: hide message
    if os.path.isdir(dirpath):
        if mode:
            print("mkdir: cannot create directory '{}': directory has already existed.".format(dirpath))
        return
    ### recursively make directory
    _temp = []
    while not os.path.isdir(dirpath):
        _temp.append(dirpath)
        dirpath = os.path.dirname(dirpath)
    while _temp:
        dirpath = _temp.pop()
        print("mkdir", dirpath)
        os.mkdir(dirpath)

def movedir(dirpath, targetdir, dirname):
    ### dirname can be different from basename of dirpath, can be used to rename a directory.
    makedir(targetdir)
    print("mv", dirpath, os.path.join(targetdir, dirname))
    shutil.move(dirpath, os.path.join(targetdir, dirname))

def savecopy(filepath, targetdir, filename):
    ### filename can be different from basename of filepath, can be used to rename a file.
    makedir(targetdir)
    print("cp -p", filepath, os.path.join(targetdir, filename))
    shutil.copy2(filepath, os.path.join(targetdir, filename))  # reserve modified date and some meta data

def savemove(filepath, targetdir, filename):
    ### filename can be different from basename of filepath, can be used to rename a file.
    makedir(targetdir)
    print("mv", filepath, os.path.join(targetdir, filename))
    shutil.move(filepath, os.path.join(targetdir, filename))

def delete(filepath):
    if not os.path.exists(filepath):
        print("rm: '{}': No such file or directory".format(filepath))
        return
    print("rm", filepath)
    os.remove(filepath)

def deletedir_empty(filepath):
    if not os.path.exists(filepath):
        print("rmdir: '{}': No such file or directory".format(filepath))
        return
    print("rmdir", filepath)
    os.rmdir(filepath)

def deletedir_nonempty(filepath):
    if not os.path.exists(filepath):
        print("rm -rf: '{}': No such file or directory".format(filepath))
        return
    print("rm -rf", filepath)
    shutil.rmtree(filepath, ignore_errors=True)

In [3]:
# ********************* User Settings *********************
database = "/home/wmnlab/D/database/"
date = "2022-10-20"
N = 8  # rounds of this experiment
Exp_Name = [  # every experiment name should start with '_' (underline)
    "_Bandlock_Udp",
    "_Bandlock_Tcp",
    # "_Udp_Stationary_Bandlock", 
    # "_Udp_Stationary_SameSetting",
]
Exp_Rounds = [
    ["#01", "#02", "#03", "#04",],
    ["#05", "#06", "#07", "#08",],
]
devices = sorted([
    # "sm03",
    "sm04",
    "sm05",
    "sm06",
    "sm07",
    "sm08",
])
delete_after_copy = False
items_copy = [  # needs to make multiple copies
    # "at_log",
    # "cimon",
    # "mi2log",
]
items_move = [  # only needs to move files
    # "at_log",
    # "cimon",
    "mi2log",
    # "client_log",
    "client_pcap",
    # "client_stats",
    # "server_log",
    "server_pcap",
    "server_stats",
]
# *********************************************************

# --------------------- Parameters ---------------------
db_path = os.path.join(database, date)
raw_path = os.path.join(db_path, "raw_data")
all_dirs = [os.path.join(raw_path, item) for item in ["at_log", "cimon", "mi2log", "client_log", "client_pcap", "client_stats", "server_log", "server_pcap", "server_stats"]]
dirs_copy = [os.path.join(raw_path, item) for item in items_copy]
dirs_move = [os.path.join(raw_path, item) for item in items_move]

# --------------------- Initialization ---------------------
makedir(db_path)
fsettings = os.path.join(db_path, "experiment-settings.txt")
if not os.path.exists(fsettings):
    print("touch", fsettings)
    open(fsettings, mode='x', encoding="utf-8")

makedir(raw_path)
for dir in all_dirs:
    makedir(dir)

print("raw_path:", raw_path)
print("dirs_copy:")
pprint(dirs_copy)
print("dirs_move:")
pprint(dirs_move)

mkdir /home/wmnlab/D/database/examples
touch /home/wmnlab/D/database/examples/experiment-settings.txt
mkdir /home/wmnlab/D/database/examples/raw_data
mkdir /home/wmnlab/D/database/examples/raw_data/at_log
mkdir /home/wmnlab/D/database/examples/raw_data/cimon
mkdir /home/wmnlab/D/database/examples/raw_data/mi2log
mkdir /home/wmnlab/D/database/examples/raw_data/client_log
mkdir /home/wmnlab/D/database/examples/raw_data/client_pcap
mkdir /home/wmnlab/D/database/examples/raw_data/client_stats
mkdir /home/wmnlab/D/database/examples/raw_data/server_log
mkdir /home/wmnlab/D/database/examples/raw_data/server_pcap
mkdir /home/wmnlab/D/database/examples/raw_data/server_stats
raw_path: /home/wmnlab/D/database/examples/raw_data
dirs_copy:
[]
dirs_move:
['/home/wmnlab/D/database/examples/raw_data/mi2log',
 '/home/wmnlab/D/database/examples/raw_data/client_pcap',
 '/home/wmnlab/D/database/examples/raw_data/server_pcap',
 '/home/wmnlab/D/database/examples/raw_data/server_stats']


In [3]:
# --------------------- Stage 0: Check Files ---------------------
print(raw_path)
for item, dir in zip(items_copy, dirs_copy):
    files = sorted(os.listdir(dir))  # default: reverse=False 按字典升冪排列
    print("|_____")
    print("      ", item, len(files))
    pprint(files)
for item, dir in zip(items_move, dirs_move):
    files = sorted(os.listdir(dir))  # default: reverse=False 按字典升冪排列
    print("|_____")
    print("      ", item, len(files))
    pprint(files)

/home/wmnlab/D/database/2022-10-20/raw_data
|_____
       mi2log 10
['diag_log_sm04_2022-10-20_13-31-05.mi2log',
 'diag_log_sm04_2022-10-20_14-23-50.mi2log',
 'diag_log_sm05_2022-10-20_13-31-05.mi2log',
 'diag_log_sm05_2022-10-20_14-23-50.mi2log',
 'diag_log_sm06_2022-10-20_13-31-05.mi2log',
 'diag_log_sm06_2022-10-20_14-23-50.mi2log',
 'diag_log_sm07_2022-10-20_13-31-06.mi2log',
 'diag_log_sm07_2022-10-20_14-23-50.mi2log',
 'diag_log_sm08_2022-10-20_13-31-06.mi2log',
 'diag_log_sm08_2022-10-20_14-23-50.mi2log']
|_____
       client_pcap 10
['client_pcap_BL_sm04_3208_3209_2022-10-20_13-30-49.pcap',
 'client_pcap_BL_sm04_3208_3209_2022-10-20_14-23-32.pcap',
 'client_pcap_BL_sm05_3210_3211_2022-10-20_13-30-51.pcap',
 'client_pcap_BL_sm05_3210_3211_2022-10-20_14-23-33.pcap',
 'client_pcap_BL_sm06_3212_3213_2022-10-20_13-30-56.pcap',
 'client_pcap_BL_sm06_3212_3213_2022-10-20_14-23-35.pcap',
 'client_pcap_BL_sm07_3214_3215_2022-10-20_13-30-58.pcap',
 'client_pcap_BL_sm07_3214_3215_2022-10-

In [4]:
# --------------------- Stage 1: Categorize In Chronological Order ---------------------
### make a directory for each round of experiments
for i in range(N):
    targetdir = os.path.join(raw_path, "#{:02d}".format(i+1))
    makedir(targetdir)
    # if os.path.isdir(targetdir):
    #     os.rmdir(targetdir)

### copy files
for item, dir in zip(items_copy, dirs_copy):
    files = sorted(os.listdir(dir))
    print("|_____")
    print("      ", item, "({})".format(len(files)))
    for i, dev in enumerate(devices):
        filename = files[i]
        for j in range(N):
            filepath = os.path.join(dir, filename)
            targetdir = os.path.join(raw_path, "#{:02d}".format(j+1), dev, "raw")
            # print("cp -p", filepath, os.path.join(targetdir, filename))
            savecopy(filepath, targetdir, filename)
        if delete_after_copy:
            # print("rm", filepath)
            delete(filepath)

### move files
M = N * len(devices)
for item, dir in zip(items_move, dirs_move):
    _files = sorted(os.listdir(dir))
    print("|_____")
    print("      ", item, "({})".format(len(_files)))
    L = len(_files) // M
    for k in range(L):
        files = _files[k*M : (k+1)*M]
        for i, dev in enumerate(devices):
            for j in range(N):
                filename = files[i*N+j]
                filepath = os.path.join(dir, filename)
                targetdir = os.path.join(raw_path, "#{:02d}".format(j+1), dev, "raw")
                # print("mv", filepath, os.path.join(targetdir, filename))
                savemove(filepath, targetdir, filename)

mkdir /home/wmnlab/D/database/2022-10-20/raw_data/#01
mkdir /home/wmnlab/D/database/2022-10-20/raw_data/#02
|_____
       mi2log (10)
mkdir /home/wmnlab/D/database/2022-10-20/raw_data/#01/sm04
mkdir /home/wmnlab/D/database/2022-10-20/raw_data/#01/sm04/raw
mv /home/wmnlab/D/database/2022-10-20/raw_data/mi2log/diag_log_sm04_2022-10-20_13-31-05.mi2log /home/wmnlab/D/database/2022-10-20/raw_data/#01/sm04/raw/diag_log_sm04_2022-10-20_13-31-05.mi2log
mkdir /home/wmnlab/D/database/2022-10-20/raw_data/#02/sm04
mkdir /home/wmnlab/D/database/2022-10-20/raw_data/#02/sm04/raw
mv /home/wmnlab/D/database/2022-10-20/raw_data/mi2log/diag_log_sm04_2022-10-20_14-23-50.mi2log /home/wmnlab/D/database/2022-10-20/raw_data/#02/sm04/raw/diag_log_sm04_2022-10-20_14-23-50.mi2log
mkdir /home/wmnlab/D/database/2022-10-20/raw_data/#01/sm05
mkdir /home/wmnlab/D/database/2022-10-20/raw_data/#01/sm05/raw
mv /home/wmnlab/D/database/2022-10-20/raw_data/mi2log/diag_log_sm05_2022-10-20_13-31-05.mi2log /home/wmnlab/D/data

In [6]:
# --------------------- Stage 2: Categorize With Experiment Names ---------------------
for j, exp in enumerate(Exp_Name):
    print("|_____")
    print("      ", exp, "({})".format(len(Exp_Rounds[j])))
    for i, _round in enumerate(Exp_Rounds[j]):
        for dev in devices:
            dirpath = os.path.join(raw_path, _round, dev, "raw")
            targetdir = os.path.join(db_path, exp, dev, "#{:02d}".format(i+1))
            # print("mv", dirpath, os.path.join(targetdir, dirname))
            movedir(dirpath, targetdir, "raw")
        deletedir_empty(os.path.join(raw_path, _round))

|_____
       _Udp_Stationary_Bandlock (1)
mv /home/wmnlab/D/database/2022-10-20/raw_data/#01/sm04 /home/wmnlab/D/database/2022-10-20/_Udp_Stationary_Bandlock/#01/sm04


FileNotFoundError: [Errno 2] No such file or directory: '/home/wmnlab/D/database/2022-10-20/raw_data/#01/sm04'

### Patch: from old to new file structure

In [16]:
import os
import sys
import shutil
from genericpath import exists
from pprint import pprint

# ********************* User Settings *********************
# database = "/home/wmnlab/Desktop/testspace/bandlock_analysis_0930/"
date = "2022-10-11"
database = "/home/wmnlab/D/database/"
# date = "2022-10-20"
db_path = os.path.join(database, date)
Exp_Name = [  # every experiment name should start with '_' (underline)
    "_Bandlock_Udp",
    "_Bandlock_Tcp",
    # "_Udp_Stationary_Bandlock", 
    # "_Udp_Stationary_SameSetting",
]
Exp_Rounds = [
    # ["#01",],
    # ["#02", "#03", "#04", "#05", "#06",],
    # ["#01", "#02", "#03", "#04", "#05", "#06",],
    ["#01", "#02", "#03", "#04",],
    ["#01", "#02", "#03", "#04",],
]
devices = sorted([
    "sm03",
    "sm04",
    "sm05",
    "sm06",
    "sm07",
    "sm08",
])
# *********************************************************

# --------------------- Patch Stage: Categorize With Experiment Names ---------------------
for j, exp in enumerate(Exp_Name):
    print("|_____")
    print("      ", exp, "({})".format(len(Exp_Rounds[j])))
    for i, _round in enumerate(Exp_Rounds[j]):
        for dev in devices:
            # dirpath = os.path.join(db_path, exp, _round, dev, "raw")
            # targetdir = os.path.join(db_path, exp, dev, _round)
            # # print("mv", dirpath, os.path.join(targetdir, "raw"))
            # movedir(dirpath, targetdir, "raw")
            # dirpath = os.path.join(db_path, exp, _round, dev, "data")
            # targetdir = os.path.join(db_path, exp, dev, _round)
            # # print("mv", dirpath, os.path.join(targetdir, "data"))
            # movedir(dirpath, targetdir, "data")
            dirpath = os.path.join(db_path, exp, _round, dev, "analysis")
            targetdir = os.path.join(db_path, exp, dev, _round)
            # print("mv", dirpath, os.path.join(targetdir, "analysis"))
            movedir(dirpath, targetdir, "analysis")
        # deletedir_nonempty(os.path.join(db_path, exp, _round))

|_____
       _Bandlock_Udp (4)
mv /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/#01/sm03/analysis /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm03/#01/analysis
mv /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/#01/sm04/analysis /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm04/#01/analysis
mv /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/#01/sm05/analysis /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm05/#01/analysis
mv /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/#01/sm06/analysis /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm06/#01/analysis
mv /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/#01/sm07/analysis /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm07/#01/analysis
mv /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/#01/sm08/analysis /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm08/#01/analysis
mv /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/#02/sm03/analysis /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm03/#02/analysis
mv /home/

#### Delete the original folder (must check if there is empty by yourself)

In [17]:
# --------------------- Patch Stage: Categorize With Experiment Names ---------------------
for j, exp in enumerate(Exp_Name):
    print("|_____")
    print("      ", exp, "({})".format(len(Exp_Rounds[j])))
    for i, _round in enumerate(Exp_Rounds[j]):
        deletedir_nonempty(os.path.join(db_path, exp, _round))

|_____
       _Bandlock_Udp (4)
rm -rf /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/#01
rm -rf /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/#02
rm -rf /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/#03
rm -rf /home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/#04
|_____
       _Bandlock_Tcp (4)
rm -rf /home/wmnlab/D/database/2022-10-11/_Bandlock_Tcp/#01
rm -rf /home/wmnlab/D/database/2022-10-11/_Bandlock_Tcp/#02
rm -rf /home/wmnlab/D/database/2022-10-11/_Bandlock_Tcp/#03
rm -rf /home/wmnlab/D/database/2022-10-11/_Bandlock_Tcp/#04
